In [ ]:
!rm -rf lstm_for_stock_price_prediction
!git clone https://github.com/Davidham3/lstm_for_stock_price_prediction.git
!pip install pytorch_lightning

In [1]:
import os
import sys
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.plugins.io import AsyncCheckpointIO
from torch import nn

# folder = "lstm_for_stock_price_prediction"
folder = "../../"
sys.path.append(os.path.join(folder, "code/model"))
from datamodule import TSDataModule
from model import LitLSTM, LSTMModel, LSTMWithAttention

/home/davidham/miniconda3/envs/tf/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = os.path.join(folder, "data/input_data.csv")
num_time_steps = 6

batch_size = 4
learning_rate = 1e-3

num_features = 9
hidden_size = 10
num_lstm_layers = 1
bidirectional = False

ts_data_module = TSDataModule(
    path,
    num_time_steps=num_time_steps,
    batch_size=batch_size
)

lstm_net = LSTMWithAttention(
    input_size=num_features,
    hidden_size=hidden_size,
    num_layers=num_lstm_layers,
    bidirectional=bidirectional,
)
loss_func = nn.MSELoss()

early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.00,
    patience=10,
    verbose=False,
    mode="min")
async_ckpt_io = AsyncCheckpointIO()
checkpoint_callback = ModelCheckpoint(
    save_top_k=3,
    monitor="val_loss",
    mode="min",
    filename="lstm-{epoch:04d}-{val_loss:.8f}",
)

(2534, 9) (634, 9) (793, 9)


/home/davidham/miniconda3/envs/tf/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:263: UserWarning: Attribute 'lstm_net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['lstm_net'])`.
  rank_zero_warn(
/home/davidham/miniconda3/envs/tf/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:263: UserWarning: Attribute 'loss_func' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_func'])`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type              | Params
------------------------------------------------
0 | lstm_net  | LSTMWithAttention | 961   
1 | loss_func | MSELoss           | 0     
--------------------------------

Sanity Checking: 0it [00:00, ?it/s]

/home/davidham/miniconda3/envs/tf/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/davidham/miniconda3/envs/tf/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  22%|██▏       | 175/789 [00:04<00:17, 35.38it/s, loss=0.0159, v_num=6]

/home/davidham/miniconda3/envs/tf/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
pl_net = LitLSTM(lstm_net, loss_func, ts_data_module.train_y_min, ts_data_module.train_y_max, learning_rate=learning_rate)
trainer = pl.Trainer(accelerator="auto", callbacks=[checkpoint_callback, early_stop_callback], max_epochs=10000, plugins=[async_ckpt_io])
trainer.fit(model=pl_net, datamodule=ts_data_module)

In [ ]:
trainer.test(pl_net, dataloaders=ts_data_module.test_dataloader())